*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  15.11.2023

--- 

# Comparison of the counting data and VISUM dtv values

<!--Notebook description and usage information-->


In [2]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

# import custom modules
sys.path.append('../utils')
from excel_calendar import Calendar
import data_paths


Loading BokehJS ...

# Import and preapre datasets

In [3]:
# import counting data
mst_file_path = data_paths.MST_COUNTING_PATH + 'preprocessed_lhm_counting_data.parquet'
bast_file_path = data_paths.BAST_COUNTING_PATH + 'preprocessed_bast_counting_data.parquet'

counting_data = pd.concat([pd.read_parquet(mst_file_path),
                           pd.read_parquet(bast_file_path)], axis=0)

# import visum_links data 
visum_file_path = data_paths.VISUM_FOLDER_PATH + 'visum_links.gpkg'
visum = gpd.read_file(visum_file_path)

# import calender
cal_obj = Calendar()

In [4]:
# aggregate and reduce to volume dataset
volume  = counting_data.groupby(['metric','road_link_id', 
                               'vehicle_class','date']).sum(numeric_only = True).loc['volume']
volume = volume.drop(['detector_id'], axis =1)
volume = volume.reset_index()

# add road type information
road_types = visum.set_index('road_link_id')['road_type'].to_dict()
volume.insert(4,'road_type' , volume['road_link_id'].map(road_types))

# add day type information
dates = volume['date'].unique()
day_types = {date:cal_obj.get_day_type(date) for date in dates}
volume.insert(5, 'day_type', volume['date'].map(day_types))

# drop rows with NaN values
volume = volume.dropna()

# reduce to 2019 counting data
volume_2019  = volume[(volume['vehicle_class'] == 'SUM') & 
                      (volume['date'].between('2019-01-01','2019-12-31'))]


In [17]:
e = 0.02
volume_sum_2019 = volume_sum_2019[volume_sum_2019.iloc[:,-25:].sum(axis=1).between(
    volume_sum_2019['daily_value']*(1-e),volume_sum_2019['daily_value']*(1+e))]

In [18]:
volume_sum_2019

,road_link_id,vehicle_class,date,daily_value,road_type,day_type,0,1,2,3,...,15,16,17,18,19,20,21,22,23,24
8965,0.000000e+00,SUM,2018-01-01,49028.0,NaN,4,1357.0,2661.0,2256.0,1540.0,...,4130.0,3842.0,3953.0,3277.0,2695.0,2272.0,1810.0,1453.0,0.0,0.0
8966,0.000000e+00,SUM,2018-01-02,78896.0,NaN,2,516.0,376.0,262.0,257.0,...,6226.0,6265.0,6137.0,5089.0,3503.0,2765.0,1972.0,1911.0,1116.0,0.0
8969,0.000000e+00,SUM,2018-01-05,84565.0,NaN,2,789.0,503.0,375.0,341.0,...,6287.0,6116.0,5971.0,5283.0,3974.0,3188.0,2556.0,2547.0,1720.0,0.0
8970,0.000000e+00,SUM,2018-01-06,69645.0,NaN,4,1177.0,845.0,568.0,471.0,...,5139.0,5187.0,5032.0,4565.0,4083.0,3333.0,2758.0,2195.0,0.0,0.0
8971,0.000000e+00,SUM,2018-01-07,59564.0,NaN,4,1222.0,987.0,745.0,606.0,...,4658.0,4706.0,4908.0,4423.0,3616.0,3038.0,2493.0,2019.0,1204.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072602,2.147483e+09,SUM,2018-12-27,23196.0,Local/Collector,2,241.0,150.0,84.0,82.0,...,1900.0,1842.0,1691.0,1499.0,1070.0,847.0,634.0,526.0,417.0,0.0
1072603,2.147483e+09,SUM,2018-12-28,23476.0,Local/Collector,2,222.0,141.0,100.0,101.0,...,1728.0,1877.0,1749.0,1455.0,1182.0,864.0,656.0,634.0,491.0,0.0
1072604,2.147483e+09,SUM,2018-12-29,19599.0,Local/Collector,3,331.0,209.0,129.0,164.0,...,1510.0,1474.0,1424.0,1339.0,981.0,815.0,603.0,553.0,478.0,0.0
1072605,2.147483e+09,SUM,2018-12-30,15419.0,Local/Collector,4,313.0,194.0,151.0,120.0,...,1312.0,1242.0,1262.0,1032.0,787.0,674.0,562.0,486.0,344.0,0.0
